<a href="https://colab.research.google.com/github/abhilashpanda04/Sentiment_Analysis-with-Bert/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.4 MB/s 
     |████████████████████████████████| 6.6 MB 44.0 MB/s 
     |████████████████████████████████| 120 kB 44.9 MB/s 


In [2]:
from transformers import BertForSequenceClassification,BertTokenizer

In [3]:
tokenizer=BertTokenizer.from_pretrained("ProsusAI/finbert")
model=BertForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [4]:
import torch

In [5]:
def sentiment(tokens):
  # get output logits from the model
  output=model(**tokens)
  probs=torch.nn.functional.softmax(output[0],dim=-1)
  return probs


In [6]:
txt='''“But for a salaried individual, the tax regime – new or existing – can be selected every year,” says Savla.

Penalty and higher taxes
Apart from paying a Rs 5,000 late filing penalty, late filers would have to pay additional tax, too, this year.

“Whatever TDS has been deducted, there could be a shortage resulting from the change in regime based on the income level of the person and the investments that have been made during the financial year,” says Ameet Patel, partner at Manohar Chowdhry & Associates.

Also read | Couldn't file a belated income-tax return? You can now use the updated tax return facility 

If there is additional tax liability, then it would have to be paid as the taxpayers would have to use the existing tax regime.

Under the new system, the threshold is not different for senior citizens and so-called super senior citizens and other individuals.

A lower rate of 10% of the income is applicable to those having income between Rs 5 lakh and Rs.7.5 lakh as against 20% under the old tax regime.

“For instance, someone who has an income of Rs 14 lakh -15 lakh and opted for the new tax regime as it was beneficial, the tax deducted at source was as per new regime rates. As the tax rates are lower for the new tax regime one would need to pay additional tax if they haven’t filed the returns within the deadline,” says Savla.

Deductions perplex
Another area of concern for those who were intending to follow the new tax regime during the financial year is that they aren’t permitted to claim 76 tax deductions and hence they did not invest to save taxes during the year.

Also read | Everything you need to know about the difference between revised, belated and updated income-tax returns

“A new tax regime would propose a lower tax rate; however, it does not permit tax deductions,” says Amit Gupta, MD, SAG Infotech.

Deductions and exemptions such as Sections 80C, 80D, Housing Rent Allowance, leave travel allowance, clubbing benefits for minors and so on aren’t permitted under the new  regime.

Many may not even have saved in the right instruments to claim deductions.

For such people, there are certain expenses that can be claimed under the belated returns that they would now have to file based on the old regime.

“If they have paid health insurance premium for self and parents, they could claim the Section 80 D benefits under the belated returns, even if the employer hasn’t been notified about the same. Additionally, school tuition fee can be claimed,” suggests Savla.

Difficulty in calculations
But since these deductions were not claimed with the employer, the Form 16 numbers wouldn’t suffice and you would have to do a fair bit of calculation to arrive at the actual taxable income.

“Those who want to now file belated returns under the existing tax regime would face serious difficulties as the Form 16 calculations are based on the new tax regime and the figures may not match,” says Patel.

So, ensure you seek help from a professional and also be on the lookout for follow-up emails from the income tax department, as the figure of your returns wouldn’t necessarily match those mentioned in the Annual Information Statement and Form 26 AS for tax deducted at source.

Select the right regime
Salaried individuals and pensioners can choose between the old tax regime and the new concessional tax regime in every assessment year based on their convenience.

However, taxpayers having business or professional income do not have the leeway to choose between the two tax regimes in every financial year. Such taxpayers will have only one option to switch between the two regimes.

“During ITR filing, you can choose either of them. But to have maximum tax savings, it is important to select suitable tax regimes at the start of the financial year,” says Gupta.'''

In [7]:
tokens=tokenizer.encode_plus(txt,add_special_tokens=False)
len(tokens["input_ids"])

Token indices sequence length is longer than the specified maximum sequence length for this model (815 > 512). Running this sequence through the model will result in indexing errors


815

In [8]:
input_ids=tokens["input_ids"]
attention_mask=tokens["attention_mask"]

In [9]:
input_ids[16:32]

[1516,
 2064,
 2022,
 3479,
 2296,
 2095,
 1010,
 1524,
 2758,
 7842,
 2615,
 2721,
 1012,
 6531,
 1998,
 3020]

In [10]:
prob_list=[]
start=0
window_size=510
total_len=len(input_ids)
loop=True
while loop:
  end=start+window_size
  if end >=total_len:
    loop= False
    end=total_len

  #extract window from input_ids and attention mask
  input_id_chunk=[101]+input_ids[start:end]+[102]
  attention_mask_chunk=[1]+attention_mask[start:end]+[1]

  input_id_chunk+=[0]*(window_size-len(input_id_chunk)+2)
  attention_mask_chunk+=[0]*(window_size-len(attention_mask_chunk)+2)

  input_dict={
      'input_ids':torch.tensor([input_id_chunk]).long(),
      'attention_mask':torch.tensor([attention_mask_chunk]).int()
    }
  prob_list.append(sentiment(input_dict))
  print(f'start={start}\n end={end}')
  start=end

start=0
 end=510
start=510
 end=815


In [11]:
len(input_id_chunk)

512

In [12]:
prob_list

[tensor([[0.0345, 0.1359, 0.8296]], grad_fn=<SoftmaxBackward0>),
 tensor([[0.0460, 0.1028, 0.8512]], grad_fn=<SoftmaxBackward0>)]

In [15]:
stack=torch.stack(prob_list)
stack

tensor([[[0.0345, 0.1359, 0.8296]],

        [[0.0460, 0.1028, 0.8512]]], grad_fn=<StackBackward0>)

In [16]:
stack.shape

torch.Size([2, 1, 3])

In [17]:
stack.shape[0],stack.shape[2]

(2, 3)

In [ ]:
stack.resize_(stack.shape[0],stack.shape[2])

In [19]:
with torch.no_grad():
  stack=torch.stack(prob_list)
  stack.resize_(stack.shape[0],stack.shape[2])
  avg=stack.mean(dim=0)

avg

tensor([0.0403, 0.1193, 0.8404])

In [20]:
import numpy as np
torch.argmax(avg).item()

2

In [ ]:
# ****other method****

In [ ]:
tokens=tokenizer.encode_plus(txt,add_special_tokens=False,return_tensors='pt')
print(len(tokens['input_ids'][0]))
tokens

In [22]:
a=torch.arange(10)
torch.split(a,4)

(tensor([0, 1, 2, 3]), tensor([4, 5, 6, 7]), tensor([8, 9]))

In [23]:
input_id_chunks=tokens['input_ids'][0].split(510)
mask_chunks=tokens['attention_mask'][0].split(510)

In [ ]:
input_id_chunks[1]

In [26]:
len(input_id_chunks[1])

305

In [27]:
a=torch.cat([torch.Tensor([101]),a,torch.Tensor([102])])

In [28]:
padding=20- a.shape[0]

padding

8

In [29]:
if padding >0:
  a=torch.cat(
      [a,torch.Tensor([0]* padding)]
  )
a

tensor([101.,   0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9., 102.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

In [41]:
chuksize=512

input_id_ch=list(tokens['input_ids'][0].split(chuksize-2))
mask_ch=list(tokens['attention_mask'][0].split(chuksize-2))

for i in range(len(input_id_ch)):
  input_id_ch[i]=torch.cat([torch.Tensor([101]),input_id_ch[i],torch.Tensor([102])])
  mask_ch[i]=torch.cat([torch.Tensor([1]),mask_ch[i],torch.Tensor([1])])

  pad_len=chuksize-input_id_ch[i].shape[0]

  if pad_len>0:
    input_id_ch[i]=torch.cat(
        [input_id_ch[i],torch.Tensor([0]*pad_len)]
    )
    mask_ch[i]=torch.cat(
        [mask_ch[i],torch.Tensor([0]*pad_len)])



In [42]:
for chunk in input_id_ch:
  print(len(chunk))

512
512


In [ ]:
chunk

In [44]:
input_ids=torch.stack(input_id_ch)
attention_mask=torch.stack(mask_ch)


In [45]:
input_dic={
    "input_ids":input_ids.long(),
    "attention_mask":attention_mask.int()
}

In [46]:
input_dic

{'input_ids': tensor([[  101,  1523,  2021,  ...,  2740,  5427,   102],
         [  101, 12882,  2005,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
         [1, 1, 1,  ..., 0, 0, 0]], dtype=torch.int32)}

In [47]:
output=model(**input_dic)
prob=torch.nn.functional.softmax(output[0],dim=-1)
prob=prob.mean(dim=0)

In [48]:
prob

tensor([0.0403, 0.1193, 0.8404], grad_fn=<MeanBackward1>)

In [52]:
torch.argmax(prob).item()

2